<a href="https://colab.research.google.com/github/kimmjen/Colab/blob/master/final_crawl_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
"""주식종목 뉴스"""

from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
import os
# 셀레니움
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from urllib.parse import quote_plus

os.chdir('./')

def crawler(company_code , maxpage):

    # url = 'https://finance.naver.com/item/news_news.nhn?code='+str(company_code) + '&page=' + str(page)
    # new_source = bs(url, 'lxml')
    # navi = new_source.find("table", class_="Nnavi")
    # navi_last = navi.find("td", class_="pgRR")
    # pag = navi_last.a.get('href').rsplit('&')[1]
    # pg_last = page.split('=')[1]
    # pg_last = int(pg_last)
    # print("총" + str(pg_last) + "개 확인")
    # maxpage = pg_last
    page = 1
    list_news = []

    while page < pg_last:
        # 뉴스제목
        titles = html.select('.title')
        title_result=[]
        for title in titles:
            title = title.get_text()
            title = re.sub('\n','',title)
            title_result.append(title)

        # 뉴스 링크
        links = html.select('.title')

        link_result = []
        for link in links:
            add = 'https://finance.naver.com'+link.find('a')['href']
            link_result.append(add)

        # 뉴스 날짜
        dates = html.select('.date')
        date_result = [date.get_text() for date in dates]

        # 뉴스 매체
        sources = html.select('.info')
        source_result = [source.get_text() for source in sources]

        # 변수들을 합쳐서 해당 디렉토리에 csv 파일로 저장하기
        results = {'날짜' : date_result, '언론사': source_result, '기사제목': title_result, '링크': link_result}
        list_news.append(results)
        # df_result = pd.DataFrame(result)

        print('다운중~')
        
        page += 1
    df_result = pd.DataFrame(list_news)
    df_result.to_csv('./' + 'news_'+ company_code +'.csv', encoding='utf-8')

# def convert_to_code(company, maxpage):

#     data = pd.read_csv('company_list.csv', dtype=str, sep='\t')
#     company_name = data['회사명']
#     keys = [i for i in company_name]

#     company_code = data['종목코드']
#     values = [j for j in company_code]

#     dict_result = dict(zip(keys, values))

#     pattern = '[0-9a-zA-Z가-힣]+'

#     if bool(re.match(pattern, company)) == True:
#         company_code = dict_result.get(str(company))
#         crawler(company_code, maxpage)

#     else:
#         company_code = str(company)
#         crawler(company_code, maxpage)

def main():
    info_main = input("="*50 + "\n" + "실시간 뉴스기사 다운받기" + "\n" + " 시작하시려면 Enter를 눌러주세요." + "\n" + "="*50)

    company_code = input("종목이나 이름이나 코드 입력 : ")

    url = 'https://finance.naver.com/item/news_news.nhn?code='+str(company_code)
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    mydata = driver.find_element_by_class_name('type5')
    mydata = driver.page_source

    html = bs(mydata, 'lxml')
    # 크롤링할 페이지 마지막페이지 확인
    navi = html.find("table", class_="Nnavi")
    navi_last = navi.find("td", class_="pgRR")
    pag = navi_last.a.get('href').rsplit('&')[1]
    pg_last = pag.split('=')[1]
    pg_last = int(pg_last)
    print("총 " + str(pg_last) + " 개 확인")
    
    maxpage = input("총 페이지 수 : {page}".format(page=pg_last))

    # convert_to_code(company, maxpage)
main()
